In [1]:
import numpy as np


In [42]:
class Lattice1D:
    def __init__(self, N, a, offsets):
        self.createLattice1d(N, a, offsets)
        


    def createLattice1d(self, N, a, offsets):
        offsets = [offsets]
        self.realLattice = np.array([a * n for n in range(N)])
        temp = []
        for i in range(N):
            for j in range(len(offsets)):
                temp.append(self.realLattice[i] + offsets[j])
        self.lattice = temp
        
    def getRealLattice(self):
        return self.realLattice
    
    def getlattice(self):
        return self.lattice

In [43]:
lat = Lattice1D(3, 1, [0, 0.8])
print(lat.getRealLattice())
print(lat.getlattice())

[0 1 2]
[array([0. , 0.8]), array([1. , 1.8]), array([2. , 2.8])]
